In [2]:
import stockml as st
from stockml import dataset as st_dt
from stockml import optimizations as st_op
from stockml.optimizations import select_features
import warnings
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import date,datetime

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')

#cd C:\Users\eckz9\Downloads\Compressed\ingw
#bin\run.bat root\conf.yaml


import subprocess

Change directory
subprocess.run(['cd', 'C:\\Users\\eckz9\\Downloads\\Compressed\\ingw'], shell=True)

Run the batch file
subprocess.run(['bin\\run.bat', 'root\\conf.yaml'], shell=True)

In [3]:
#Call the Interactive Broker API after establising a connection above in order to get a dataframe of assets (along with other relevant information) available in ibapi
con_list = st.interactive_brokers.pull_ib_contract_list()
con_list.head(20)

,con_id,company_name,scan_data,last_price,listing_exchange,sec_type
symbol,,,,,,
SPY,756733,SPDR S&P 500 ETF TRUST,5.910M,1.1510,ARCA,STK
QQQ,320227571,INVESCO QQQ TRUST SERIES 1,3.483M,1.4247,NASDAQ.NMS,STK
IWM,9579970,ISHARES RUSSELL 2000 ETF,1.148M,1.0515,ARCA,STK
TQQQ,72539702,PROSHARES ULTRAPRO QQQ,285.711K,0.8635,NASDAQ.NMS,STK
SOXL,73340487,DIREXION DAILY SEMI BULL 3X,169.939K,0.4185,ARCA,STK
XLF,4215220,FINANCIAL SELECT SECTOR SPDR,162.904K,0.4720,ARCA,STK
EWZ,10753244,ISHARES MSCI BRAZIL ETF,162.761K,0.4362,ARCA,STK
ARKK,172522644,ARK INNOVATION ETF,146.739K,0.4279,ARCA,STK
FXI,31421120,ISHARES CHINA LARGE-CAP ETF,144.247K,0.4436,ARCA,STK


In [4]:
#Gather metadata about a specific stock using the ticker and the contract ID from the and storing it afterwards on a separate sql table

con_details = st.contract_details(con_list.loc['SPY', 'con_id'])
st.write_stock_metadata(con_details, st.Stocks)

Stock metadata written successfully.


In [5]:
#Get all of the rows from a specific SQL table, here we will look at the Stock metadata table
stocks_df = st.get_all_rows(st.Stocks)
stocks_df

,ticker,conid,currency,listing_exchange,country_code,name,asset_class,group,sector,sector_group,type,has_options
0,UDOW,72539713,USD,ARCA,US,PROSHARES ULTRAPRO DOW30,STK,None,None,None,ETF,True
1,SMH,229725622,USD,NASDAQ,US,VANECK SEMICONDUCTOR ETF,STK,None,None,None,ETF,True
2,UPRO,61228752,USD,ARCA,US,PROSHARES ULTRAPRO S&P 500,STK,None,None,None,ETF,True
3,FAS,97276826,USD,ARCA,US,DIREXION DAILY FIN BULL 3X,STK,None,None,None,ETF,True
4,XLY,4215215,USD,ARCA,US,CONSUMER DISCRETIONARY SELT,STK,None,None,None,ETF,True
5,XLP,4215210,USD,ARCA,US,CONSUMER STAPLES SPDR,STK,None,None,None,ETF,True
6,SPY,756733,USD,ARCA,US,SPDR S&P 500 ETF TRUST,STK,None,None,None,ETF,True


In [7]:
#Request OHCLV data from the ibapi, specifically for UPRO 
con_id = stocks_df.iat[2,1]
ticker = stocks_df.iat[2,0]

stock_data = st.pull_ib_stock_data(con_id, ticker)
stock_data

,price_id,datasource,ticker,timestamp,open,high,low,close,volume
0,UPRO ibapi 2023-12-08 01:00:00,ibapi,UPRO,2023-12-08 01:00:00,48.94,49.12,48.93,49.07,17019.0
1,UPRO ibapi 2023-12-08 01:30:00,ibapi,UPRO,2023-12-08 01:30:00,49.07,49.08,49.06,49.08,1260.0
2,UPRO ibapi 2023-12-08 02:00:00,ibapi,UPRO,2023-12-08 02:00:00,49.10,49.13,49.06,49.13,8200.0
3,UPRO ibapi 2023-12-08 02:30:00,ibapi,UPRO,2023-12-08 02:30:00,49.15,49.17,49.14,49.14,7522.0
4,UPRO ibapi 2023-12-08 03:00:00,ibapi,UPRO,2023-12-08 03:00:00,49.14,49.14,49.08,49.08,4579.0
...,...,...,...,...,...,...,...,...,...
8013,UPRO ibapi 2024-12-06 14:30:00,ibapi,UPRO,2024-12-06 14:30:00,99.42,99.42,99.42,99.42,0.0
8014,UPRO ibapi 2024-12-06 15:00:00,ibapi,UPRO,2024-12-06 15:00:00,99.39,99.39,99.20,99.20,3000.0
8015,UPRO ibapi 2024-12-06 15:30:00,ibapi,UPRO,2024-12-06 15:30:00,99.20,99.20,99.20,99.20,100.0
8016,UPRO ibapi 2024-12-06 16:00:00,ibapi,UPRO,2024-12-06 16:00:00,99.15,99.18,99.15,99.15,1555.0


In [7]:
#Write the OHLCV data into a Stock Price SQL Table
st.write_historical_data(stock_data, st.StockPrices30m)

Historical data written successfully.


In [ ]:
#Get all of the historical data that we have stored for UPRO from the stock historical data table
stock_ml_df = st.get_filtered_data(st.StockPrices30m, 'ibapi', 'UPRO')
stock_ml_df

,price_id,datasource,ticker,timestamp,open,high,low,close,volume
0,UPRO ibapi 2023-10-16 01:00:00,ibapi,UPRO,2023-10-16 01:00:00,42.19,42.22,41.99,42.18,30851.0
1,UPRO ibapi 2023-10-16 01:30:00,ibapi,UPRO,2023-10-16 01:30:00,42.26,42.39,42.26,42.36,2474.0
2,UPRO ibapi 2023-10-16 02:00:00,ibapi,UPRO,2023-10-16 02:00:00,42.30,42.36,42.28,42.30,3900.0
3,UPRO ibapi 2023-10-16 02:30:00,ibapi,UPRO,2023-10-16 02:30:00,42.29,42.39,42.29,42.31,3300.0
4,UPRO ibapi 2023-10-16 03:00:00,ibapi,UPRO,2023-10-16 03:00:00,42.24,42.27,42.17,42.27,1792.0
...,...,...,...,...,...,...,...,...,...
8769,UPRO ibapi 2024-11-15 08:00:00,ibapi,UPRO,2024-11-15 08:00:00,89.93,90.35,89.84,90.29,52338.0
8770,UPRO ibapi 2024-11-15 08:30:00,ibapi,UPRO,2024-11-15 08:30:00,90.32,90.37,89.51,89.65,58416.0
8771,UPRO ibapi 2024-11-15 09:00:00,ibapi,UPRO,2024-11-15 09:00:00,89.72,89.72,89.38,89.44,49917.0
8772,UPRO ibapi 2024-11-15 09:30:00,ibapi,UPRO,2024-11-15 09:30:00,89.43,89.66,89.14,89.43,70932.0


In [ ]:
#Validate the data before calculating technical indicators by removing uneeded columns
ohlcv_df = st.validate(stock_ml_df)
print(ohlcv_df.duplicated().all().any(), ohlcv_df.isna().all().any())
print('\n')

#Calculate all of the technical indicators available in the pandas_ta library, store all the dataframes (one for each indicator category) in a dictionary
stock_tas = st.get_all_pandas_ta(ohlcv_df)

False False




3it [00:02,  1.15it/s]
1it [00:01,  1.07s/it]
39it [00:03, 12.12it/s]
32it [00:02, 13.06it/s]
2it [00:00,  2.37it/s]
10it [00:00, 11.18it/s]
14it [00:01,  9.98it/s]
14it [00:01, 12.80it/s]
14it [00:00, 16.08it/s]


In [114]:
%store stock_tas

Stored 'stock_tas' (dict)


In [10]:
#Verify that all dataframes have the same length 
for key in stock_tas.keys():
    print(key, len(stock_tas[key]))

df1 8573
df2 8573
df3 8573
df4 8573
df5 8573
df6 8573
df7 8573
df8 8573
df9 8573
df0 8573


In [ ]:
#Generate target variables (such as percent change in close or difference in closing prices, which are both shifted) using a dataframe that does not contain any technical indicators
X_df = stock_tas['df0']
y_df = st.generate_target_vars(st.validate(stock_ml_df[200:]))
y_df.head()

,open,high,low,close,volume,low_shift,open_shift,drop_shift,drop_percent_shift,high_shift,peak_shift,peak_percent_shift,change,high_diff,low_diff,diff,diff_shift,change_shift,close_shift
timestamp,,,,,,,,,,,,,,,,,,,
2023-10-24 05:00:00,39.36,39.49,39.14,39.45,91833.0,39.38,39.45,-0.07,-0.177440,39.50,0.05,0.126743,NaN,NaN,NaN,NaN,0.01,0.025349,39.46
2023-10-24 05:30:00,39.45,39.50,39.38,39.46,31439.0,39.27,39.47,-0.20,-0.506714,39.47,0.00,0.000000,0.025349,0.01,0.24,0.01,-0.14,-0.354790,39.32
2023-10-24 06:00:00,39.47,39.47,39.27,39.32,78226.0,39.27,39.32,-0.05,-0.127162,39.67,0.35,0.890132,-0.354790,-0.03,-0.11,-0.14,0.33,0.839268,39.65
2023-10-24 06:30:00,39.32,39.67,39.27,39.65,823876.0,39.30,39.65,-0.35,-0.882724,39.83,0.18,0.453972,0.839268,0.20,0.00,0.33,0.17,0.428752,39.82
2023-10-24 07:00:00,39.65,39.83,39.30,39.82,745820.0,39.66,39.82,-0.16,-0.401808,39.93,0.11,0.276243,0.428752,0.16,0.03,0.17,-0.07,-0.175791,39.75


In [13]:
#Using change_shift (which represents the percent change of closing prices) as the target variable, we use a function that displays the distrbutions of our target and outputs a numerical summary of this distribution
describe_dict = st.display_and_describe(y_df[['change_shift']])

,change_shift
nobs,8573.000000
missing,0.000000
mean,0.010355
std_err,0.004326
upper_ci,0.018834
lower_ci,0.001876
std,0.400551
iqr,0.301331
iqr_normal,0.223377
mad,0.246575


In [145]:
#Now in order to turn this into a classification problem, we will encode change_shift to 3 labels that represents either significant positive difference (buy: 1), significant negative difference (sell: -1), or somewhere in between (no position: 0). By using the 33% and 66% percentile as boundaries, we can make equal number of samples between each label.  
from stockml.dataset import eda
y_copy = eda.categorize_instances(y_df, 0.3333, 0.6666)
y_copy

Lower bound value:-0.0733//Upper bound value:0.1008


,change_shift,diff_shift,change_encoded
timestamp,,,
2023-10-24 05:00:00,0.025349,0.01,0
2023-10-24 05:30:00,-0.354790,-0.14,-1
2023-10-24 06:00:00,0.839268,0.33,1
2023-10-24 06:30:00,0.428752,0.17,1
2023-10-24 07:00:00,-0.175791,-0.07,-1
...,...,...,...
2024-11-15 07:30:00,0.444988,0.40,1
2024-11-15 08:00:00,-0.708827,-0.64,-1
2024-11-15 08:30:00,-0.234244,-0.21,-1


In [ ]:
#Now, we verify that all of the beggining and ending timestamps are the same for all dfs
for key in stock_tas.keys():
    print(key, stock_tas[key].index[0], stock_tas[key].index[-1])
    
print(y_copy.index[0], y_copy.index[-1])

df1 2023-10-24 05:00:00 2024-11-15 09:30:00
df2 2023-10-24 05:00:00 2024-11-15 09:30:00
df3 2023-10-24 05:00:00 2024-11-15 09:30:00
df4 2023-10-24 05:00:00 2024-11-15 09:30:00
df5 2023-10-24 05:00:00 2024-11-15 09:30:00
df6 2023-10-24 05:00:00 2024-11-15 09:30:00
df7 2023-10-24 05:00:00 2024-11-15 09:30:00
df8 2023-10-24 05:00:00 2024-11-15 09:30:00
df9 2023-10-24 05:00:00 2024-11-15 09:30:00
df0 2023-10-24 05:00:00 2024-11-15 09:30:00
2023-10-24 05:00:00 2024-11-15 09:30:00


In [16]:
test_size = 1000
importance = 0.05

#Initiate a dataframe that contains all of the technical indicators which will be used in the lines below
merged_df = pd.concat(stock_tas.values(), axis=1)
df_cleaned = merged_df.loc[:, ~merged_df.columns.duplicated()]

#Apply feature selection through L1 regularization first which will shrink the coefficients of insignificant features to almost zero, then subsequently filter further by leveraging Random Forest's feature importances. 
l1_rf_selected_df = select_features.select_l1_rf_features(stock_tas, y_copy[['change_encoded']], df_cleaned, importance=importance, test_size=test_size)
l1_rf_selected_df

Performing L1-based feature selection
L1 selection results


df1 - Number of features chosen by L1: 63, Number of starting features: 75
df2 - Number of features chosen by L1: 4, Number of starting features: 6
df3 - Number of features chosen by L1: 67, Number of starting features: 76
df4 - Number of features chosen by L1: 13, Number of starting features: 45
df5 - Number of features chosen by L1: 4, Number of starting features: 7
df6 - Number of features chosen by L1: 17, Number of starting features: 21
df7 - Number of features chosen by L1: 21, Number of starting features: 28
df8 - Number of features chosen by L1: 20, Number of starting features: 37
df9 - Number of features chosen by L1: 20, Number of starting features: 24


Performing Random Forest-based feature selection on the L1-selected features
Feature ranking:
1. feature no:2 feature name:volume (0.191723)
2. feature no:59 feature name:low_Z_30_1 (0.087154)
3. feature no:60 feature name:close_Z_30_1 (0.078090)
4. feature no:58 fe

,open_Z_30_1,close_Z_30_1,DMN_14,EBSW_40_10,ZS_30,VHF_28,QS_10,HA_low,TRUERANGE_1,BBB_5_2.0,...,EFI_13,PVOL,KURT_30,VTXP_14,UI_14,NATR_14,ALMA_10_6.0_0.85,KVOs_34_55_13,MASSI_9_25,high
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-24 05:00:00,0.749747,1.058217,25.195971,0.978588,1.058217,0.343849,0.048,39.14,0.35,0.647137,...,-8486.457199,3622811.85,0.045667,1.047337,1.123455,0.497720,39.194224,5985.495134,21.418169,39.49
2023-10-24 05:30:00,1.076563,1.039700,24.064652,0.969905,1.039700,0.359736,0.044,39.38,0.12,0.456346,...,-7229.193313,1240582.94,0.087632,1.168539,0.990623,0.483774,39.245974,6437.585538,21.430027,39.50
2023-10-24 06:00:00,1.097905,0.389997,26.371830,0.686230,0.389997,0.378472,0.030,39.27,0.20,0.571736,...,-7760.971411,3075846.32,0.360126,1.108247,0.826372,0.487150,39.295096,6654.221802,21.418282,39.47
2023-10-24 06:30:00,0.400556,1.709097,22.721894,0.683640,1.709097,0.351613,0.049,39.27,0.40,1.079222,...,32187.607362,32666683.40,0.196549,1.090517,0.610504,0.520648,39.329201,9443.775772,21.589385,39.67
2023-10-24 07:00:00,1.726347,2.166695,18.974597,0.675432,2.166695,0.333333,0.055,39.30,0.53,1.771948,...,45702.149167,29698552.40,0.038819,1.086643,0.345616,0.576465,39.360577,13955.634342,22.009362,39.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 07:30:00,-2.378079,-2.337215,51.905283,-0.962727,-2.337215,0.609859,-0.189,89.76,0.70,3.391148,...,-39759.020105,14402355.58,-0.207709,0.545620,1.976385,0.476131,92.018770,-3327.400153,23.252898,90.46
2024-11-15 08:00:00,-2.301847,-1.818870,47.547030,-0.990263,-1.818870,0.581208,-0.167,89.84,0.51,2.682081,...,-31088.417233,4725598.02,-0.370601,0.611670,1.923261,0.480509,92.041868,-3442.752940,23.541289,90.35
2024-11-15 08:30:00,-1.795842,-2.135665,46.333128,-0.980181,-2.135665,0.609333,-0.234,89.51,0.86,1.925388,...,-31988.106199,5236994.40,-0.336693,0.599647,1.927163,0.517893,91.986904,-3547.424679,23.727980,90.37


In [17]:
#Get a separate set of features by using a different feature selection method that comes from a library called mRMR (which stands for minimum Redundancy - Maximum Relevance https://github.com/smazzanti/mrmr)
mrmr_selected_df = st_op.select_features.select_mrmr_features(stock_tas, y_copy[['change_encoded']], df_cleaned, K=5, test_size=test_size)

mrmr_selected_df

100%|██████████| 5/5 [00:02<00:00,  2.42it/s]


,EBSW_40_10,PVOh_12_26_9,CDL_THRUSTING,ZS_30,TRUERANGE_1,low,SQZPRO_ON_NARROW,HILOs_13_21,HILOl_13_21,PDIST,...,PCTRET_1,CDL_TAKURI,PVOL,KURT_30,VTXP_14,AROOND_14,open,MASSI_9_25,high,PSARr_0.02_0.2
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-24 05:00:00,0.978588,-6.326469,0.0,1.058217,0.35,39.14,0,0.000000,39.073333,0.68,...,0.000507,0.0,3622811.85,0.045667,1.047337,50.000000,39.36,21.418169,39.49,0
2023-10-24 05:30:00,0.969905,-6.188582,0.0,1.039700,0.12,39.38,0,0.000000,39.084286,0.23,...,0.000253,0.0,1240582.94,0.087632,1.168539,42.857143,39.45,21.430027,39.50,1
2023-10-24 06:00:00,0.686230,-2.751633,0.0,0.389997,0.20,39.27,0,0.000000,39.088095,0.26,...,-0.003548,0.0,3075846.32,0.360126,1.108247,35.714286,39.47,21.418282,39.47,0
2023-10-24 06:30:00,0.683640,35.211529,0.0,1.709097,0.40,39.27,0,0.000000,39.107143,0.47,...,0.008393,0.0,32666683.40,0.196549,1.090517,28.571429,39.32,21.589385,39.67,0
2023-10-24 07:00:00,0.675432,44.326974,0.0,2.166695,0.53,39.30,0,0.000000,39.136667,0.89,...,0.004288,0.0,29698552.40,0.038819,1.086643,21.428571,39.65,22.009362,39.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15 07:30:00,-0.962727,38.231062,0.0,-2.337215,0.70,89.76,0,91.772308,0.000000,1.16,...,-0.002663,0.0,14402355.58,-0.207709,0.545620,100.000000,90.13,23.252898,90.46,0
2024-11-15 08:00:00,-0.990263,27.167795,0.0,-1.818870,0.51,89.84,0,91.662308,0.000000,0.70,...,0.004450,0.0,4725598.02,-0.370601,0.611670,92.857143,89.93,23.541289,90.35,0
2024-11-15 08:30:00,-0.980181,19.392946,0.0,-2.135665,0.86,89.51,0,91.560000,0.000000,1.08,...,-0.007088,0.0,5236994.40,-0.336693,0.599647,100.000000,90.32,23.727980,90.37,0


In [116]:
%store l1_rf_selected_df
%store mrmr_selected_df

Stored 'l1_rf_selected_df' (DataFrame)
Stored 'mrmr_selected_df' (DataFrame)


In [18]:
from stockml.utils.config import models as st_models
from stockml.utils.config import model_names as st_names

#We get the names and actual model instances themselves from config.py
model_dict = st_models
model_names = st_names

l1_rf_dict = {}
mrmr_dict = {}

In [19]:

#To narrow it down even further down to 5 to 10 features, we run Sequential Feature Selection on both the L1/Random Forest set and the mRMR set with the a model that is chosen from a specified list found in config files
for model in model_names:
    l1_rf_feat = select_features.run_sfs_with_model(l1_rf_selected_df, y_copy[['change_encoded']],        test_size=test_size, model = model_dict[model], k_features=(5,10))
    l1_rf_dict[model] = l1_rf_feat

for model in model_names:
    mrmr_feat = select_features.run_sfs_with_model(mrmr_selected_df, y_copy[['change_encoded']], test_size=test_size, model = model_dict[model], k_features=(5,10))
    mrmr_dict[model] = mrmr_feat

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   12.1s
[Parallel(n_jobs=4)]: Done  48 out of  48 | elapsed:   17.8s finished

[2024-11-15 11:26:52] Features: 1/10 -- score: 0.415[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done  47 out of  47 | elapsed:    5.0s finished

[2024-11-15 11:26:57] Features: 2/10 -- score: 0.439[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done  46 out of  46 | elapsed:    6.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.9s finished

[2024-11-15 11:27:05] Features: 3/10 -- score: 0.433[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Pa

Best performing subset (features): ('HA_low', 'BBB_5_2.0', 'low', 'ISA_9', 'PDIST')


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.0min
[Parallel(n_jobs=4)]: Done  48 out of  48 | elapsed:  3.1min finished

[2024-11-15 11:31:31] Features: 1/10 -- score: 0.4[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  47 out of  47 | elapsed:  1.7min finished

[2024-11-15 11:33:14] Features: 2/10 -- score: 0.405[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   59.9s
[Parallel(n_jobs=4)]: Done  46 out of  46 | elapsed:  1.3min finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   10.1s finished

[2024-11-15 11:34:44] Features: 3/10 -- score: 0.437[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   59.3s
[Parallel(n_jobs=4)

Best performing subset (features): ('VHF_28', 'TRUERANGE_1', 'volume', 'KVO_34_55_13', 'STDEV_30', 'SUPERT_7_3.0', 'ISB_26', 'VTXP_14')


[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:   10.1s finished

[2024-11-15 11:46:53] Features: 1/10 -- score: 0.415[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  34 out of  34 | elapsed:    3.8s finished

[2024-11-15 11:46:57] Features: 2/10 -- score: 0.438[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed:    4.5s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.9s finished

[2024-11-15 11:47:03] Features: 3/10 -- score: 0.439[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    6.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.9s finished

[2024-11-15 11:47:12] Features: 4/10 -- score

Best performing subset (features): ('HILOs_13_21', 'PDIST', 'MFI_14', 'SUPERTd_7_3.0', 'QQE_14_5_4.236_RSIMA', 'SUPERTl_7_3.0', 'KURT_30', 'MASSI_9_25')


[Parallel(n_jobs=4)]: Done  35 out of  35 | elapsed:  1.8min finished

[2024-11-15 11:50:11] Features: 1/10 -- score: 0.4[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  34 out of  34 | elapsed:  1.0min finished

[2024-11-15 11:51:13] Features: 2/10 -- score: 0.423[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 out of  33 | elapsed:   52.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.8s finished

[2024-11-15 11:52:10] Features: 3/10 -- score: 0.436[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:   46.3s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    6.6s finished

[2024-11-15 11:53:04] Features: 4/10 -- score: 

Best performing subset (features): ('TRUERANGE_1', 'low', 'SQZPRO_ON_NARROW', 'THERMOs_20_2_0.5', 'INC_1', 'QQE_14_5_4.236_RSIMA', 'ADOSC_3_10', 'PVOL')


[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed:   39.2s finished

[2024-11-15 12:04:51] Features: 10/10 -- score: 0.42

In [117]:
print(l1_rf_dict)
#{'hoeffding_adaptive': ['HA_low', 'BBB_5_2.0', 'low', 'ISA_9', 'PDIST'], 'arfc': ['VHF_28', 'TRUERANGE_1', 'volume', 'KVO_34_55_13', 'STDEV_30', 'SUPERT_7_3.0', 'ISB_26', 'VTXP_14']}
%store l1_rf_dict

{'hoeffding_adaptive': ['HA_low', 'BBB_5_2.0', 'low', 'ISA_9', 'PDIST'], 'arfc': ['VHF_28', 'TRUERANGE_1', 'volume', 'KVO_34_55_13', 'STDEV_30', 'SUPERT_7_3.0', 'ISB_26', 'VTXP_14']}
Stored 'l1_rf_dict' (dict)


In [118]:
print(mrmr_dict)
#{'hoeffding_adaptive': ['HILOs_13_21', 'PDIST', 'MFI_14', 'SUPERTd_7_3.0', 'QQE_14_5_4.236_RSIMA', 'SUPERTl_7_3.0', 'KURT_30', 'MASSI_9_25'], 'arfc': ['TRUERANGE_1', 'low', 'SQZPRO_ON_NARROW', 'THERMOs_20_2_0.5', 'INC_1', 'QQE_14_5_4.236_RSIMA', 'ADOSC_3_10', 'PVOL']}
%store mrmr_dict

{'hoeffding_adaptive': ['HILOs_13_21', 'PDIST', 'MFI_14', 'SUPERTd_7_3.0', 'QQE_14_5_4.236_RSIMA', 'SUPERTl_7_3.0', 'KURT_30', 'MASSI_9_25'], 'arfc': ['TRUERANGE_1', 'low', 'SQZPRO_ON_NARROW', 'THERMOs_20_2_0.5', 'INC_1', 'QQE_14_5_4.236_RSIMA', 'ADOSC_3_10', 'PVOL']}
Stored 'mrmr_dict' (dict)


In [ ]:
#We initialize certain variables one more time before running the main machine learning optimization which is done through optuna. 
model_name = model_names[0]
model_to_optimize = model_dict[model_name]
mrmr_hoeff_a_features = mrmr_dict[model_name]

X = mrmr_selected_df[mrmr_hoeff_a_features][-2000:]
y = y_copy[-2000:]

#For this Optuna run, we are optimizing a classification problem that will try to predict if the next candle will close significantly higher (buy), significantly lower (sell), or somewhere in between (no action taken). The metric will be based on how much reward each trial produces based on if the model's label prediction matches the actual value of y at each row, tracking the total rewards and total losses at each step.

#First we run a trial run in order to get an idea of which hyperparameters need to be focused on
from stockml.optimizations.model_tuning import optuna_classification_reward
study = optuna_classification_reward(X_data=X, y_data=y,model=model_name, n_trials=50)

[I 2024-11-15 12:35:21,032] A new study created in memory with name: no-name-98993dc3-0d28-466e-a7ee-1ba7438acc6b
[I 2024-11-15 12:35:23,081] Trial 0 finished with value: -1.740000000000009 and parameters: {'grace_period': 256, 'split_criterion': 'gini', 'delta': 0.0016542121847731904, 'tau': 0.04468478692742264, 'leaf_prediction': 'mc', 'nb_threshold': 2, 'bootstrap_sampling': True, 'drift_window_threshold': 70, 'switch_significance': 0.0724994588854443, 'binary_split': False, 'min_branch_fraction': 0.04701206273421999, 'max_share_to_split': 0.534680466684865, 'remove_poor_attrs': True, 'merit_preprune': False, 'adwin_delta': 0.0006170126465919922, 'adwin_clock': 34, 'adwin_max_buckets': 2, 'adwin_min_window_length': 5, 'adwin_grace_period': 13}. Best is trial 0 with value: -1.740000000000009.


Precision:Precision: 28.29%
Positive Predictions:43.0
Average Profit:0.31837209302325475//Average Loss:-0.2137179487179491


[I 2024-11-15 12:35:26,076] Trial 1 finished with value: 7.25999999999992 and parameters: {'grace_period': 168, 'split_criterion': 'info_gain', 'delta': 6.743360674967879e-07, 'tau': 0.04301088595364406, 'leaf_prediction': 'nba', 'nb_threshold': 22, 'bootstrap_sampling': False, 'drift_window_threshold': 160, 'switch_significance': 0.1915807971055171, 'binary_split': True, 'min_branch_fraction': 0.07209895675171031, 'max_share_to_split': 0.8310711931876935, 'remove_poor_attrs': True, 'merit_preprune': False, 'adwin_delta': 1.150338180637202e-05, 'adwin_clock': 54, 'adwin_max_buckets': 6, 'adwin_min_window_length': 3, 'adwin_grace_period': 36}. Best is trial 1 with value: 7.25999999999992.


Precision:Precision: 34.30%
Positive Predictions:249.0
Average Profit:0.3380722891566267//Average Loss:-0.2398830409356731


[I 2024-11-15 12:35:28,253] Trial 2 finished with value: 1.5600000000000023 and parameters: {'grace_period': 67, 'split_criterion': 'gini', 'delta': 0.006066443518039096, 'tau': 0.03600235075269683, 'leaf_prediction': 'mc', 'nb_threshold': 37, 'bootstrap_sampling': True, 'drift_window_threshold': 192, 'switch_significance': 0.015453618918830531, 'binary_split': False, 'min_branch_fraction': 0.06045328908201672, 'max_share_to_split': 0.9752974996202526, 'remove_poor_attrs': False, 'merit_preprune': True, 'adwin_delta': 2.0267002906189864e-05, 'adwin_clock': 61, 'adwin_max_buckets': 10, 'adwin_min_window_length': 18, 'adwin_grace_period': 49}. Best is trial 1 with value: 7.25999999999992.


...

Precision:Precision: 15.38%
Positive Predictions:4.0
Average Profit:0.17249999999999588//Average Loss:-0.13526315789473647


[I 2024-11-15 12:38:04,647] Trial 48 finished with value: 5.070000000000022 and parameters: {'grace_period': 226, 'split_criterion': 'gini', 'delta': 0.045562944120265306, 'tau': 0.05814966436784978, 'leaf_prediction': 'mc', 'nb_threshold': 56, 'bootstrap_sampling': False, 'drift_window_threshold': 151, 'switch_significance': 0.1900962802268854, 'binary_split': True, 'min_branch_fraction': 0.08300487534653841, 'max_share_to_split': 0.8977138912410993, 'remove_poor_attrs': False, 'merit_preprune': True, 'adwin_delta': 0.001656722999315748, 'adwin_clock': 12, 'adwin_max_buckets': 8, 'adwin_min_window_length': 18, 'adwin_grace_period': 46}. Best is trial 32 with value: 20.20000000000003.


Precision:Precision: 36.11%
Positive Predictions:52.0
Average Profit:0.4040384615384616//Average Loss:-0.27596774193548407


[I 2024-11-15 12:38:09,145] Trial 49 finished with value: 12.189999999999898 and parameters: {'grace_period': 206, 'split_criterion': 'hellinger', 'delta': 0.007360288730351925, 'tau': 0.038458145238503764, 'leaf_prediction': 'nb', 'nb_threshold': 65, 'bootstrap_sampling': True, 'drift_window_threshold': 181, 'switch_significance': 0.09675095838026312, 'binary_split': False, 'min_branch_fraction': 0.0695173972364373, 'max_share_to_split': 0.7928721551774441, 'remove_poor_attrs': True, 'merit_preprune': True, 'adwin_delta': 0.012860597245382955, 'adwin_clock': 27, 'adwin_max_buckets': 3, 'adwin_min_window_length': 14, 'adwin_grace_period': 46}. Best is trial 32 with value: 20.20000000000003.


Precision:Precision: 33.98%
Positive Predictions:244.0
Average Profit:0.34184426229508225//Average Loss:-0.21744985673352488


In [ ]:
#We display the importances of our parameters in order to see what we could focus on in our next run
import optuna
param_importances = optuna.visualization.plot_param_importances(study)
param_importances.show()

In [ ]:
#We change the values of some parameters and delete ones that had low importance before running optuna again
new_params = {'leaf_prediction':('categorical', ['nb'])
}
exclude_params = ['bootstrap_sampling', 'remove_poor_attrs', 'merit_preprune', 'binary_split']


now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M")
current_time = datetime.strptime(current_time, "%Y-%m-%d %H:%M")

#Run optuna again with the new parameter changes
study2 = optuna_classification_reward(X_data=X, 
                                      y_data=y,
                                      model=model_name, 
                                      n_trials=500, 
                                      param_ranges=new_params, exclude_params=exclude_params)


[I 2024-11-15 14:42:27,730] A new study created in memory with name: no-name-7d44340b-82df-4007-8b4f-c86d97c0d6f3
[I 2024-11-15 14:42:31,013] Trial 0 finished with value: 7.3300000000000125 and parameters: {'grace_period': 81, 'split_criterion': 'info_gain', 'delta': 8.001037404659774e-07, 'tau': 0.04503361318110867, 'leaf_prediction': 'nb', 'nb_threshold': 39, 'drift_window_threshold': 60, 'switch_significance': 0.031926143498031266, 'min_branch_fraction': 0.009825793493323957, 'max_share_to_split': 0.9921954634697456, 'adwin_delta': 1.1949434716435592e-05, 'adwin_clock': 14, 'adwin_max_buckets': 10, 'adwin_min_window_length': 11, 'adwin_grace_period': 6}. Best is trial 0 with value: 7.3300000000000125.


Precision:Precision: 34.25%
Positive Predictions:273.0
Average Profit:0.3235897435897437//Average Loss:-0.2317962466487938


[I 2024-11-15 14:42:34,147] Trial 1 finished with value: 5.94999999999996 and parameters: {'grace_period': 162, 'split_criterion': 'info_gain', 'delta': 1.5484388636828512e-05, 'tau': 0.06642499062839294, 'leaf_prediction': 'nb', 'nb_threshold': 8, 'drift_window_threshold': 84, 'switch_significance': 0.019248990684780427, 'min_branch_fraction': 0.016795556705331012, 'max_share_to_split': 0.7860704698560412, 'adwin_delta': 0.0381498588137471, 'adwin_clock': 41, 'adwin_max_buckets': 7, 'adwin_min_window_length': 6, 'adwin_grace_period': 13}. Best is trial 0 with value: 7.3300000000000125.


Precision:Precision: 34.54%
Positive Predictions:268.0
Average Profit:0.3324626865671644//Average Loss:-0.2423287671232881


[I 2024-11-15 14:42:38,545] Trial 2 finished with value: 10.519999999999968 and parameters: {'grace_period': 136, 'split_criterion': 'gini', 'delta': 0.050560420251040045, 'tau': 0.04448963014530272, 'leaf_prediction': 'nb', 'nb_threshold': 100, 'drift_window_threshold': 153, 'switch_significance': 0.15333323553163744, 'min_branch_fraction': 0.04994985945434565, 'max_share_to_split': 0.8793543884498839, 'adwin_delta': 0.004445426853705694, 'adwin_clock': 34, 'adwin_max_buckets': 6, 'adwin_min_window_length': 7, 'adwin_grace_period': 19}. Best is trial 2 with value: 10.519999999999968.


...

Precision:Precision: 34.69%
Positive Predictions:238.0
Average Profit:0.35558823529411826//Average Loss:-0.23817910447761217


[I 2024-11-15 15:14:21,744] Trial 498 finished with value: 5.4899999999999665 and parameters: {'grace_period': 153, 'split_criterion': 'gini', 'delta': 0.0558919229709913, 'tau': 0.06874564844123568, 'leaf_prediction': 'nb', 'nb_threshold': 67, 'drift_window_threshold': 124, 'switch_significance': 0.14616511082194372, 'min_branch_fraction': 0.09584867320292251, 'max_share_to_split': 0.6126016874866267, 'adwin_delta': 0.0012177894412012244, 'adwin_clock': 7, 'adwin_max_buckets': 10, 'adwin_min_window_length': 9, 'adwin_grace_period': 29}. Best is trial 100 with value: 26.520000000000024.


Precision:Precision: 34.09%
Positive Predictions:240.0
Average Profit:0.32950000000000057//Average Loss:-0.23479041916167712


[I 2024-11-15 15:14:26,428] Trial 499 finished with value: 7.29999999999994 and parameters: {'grace_period': 126, 'split_criterion': 'gini', 'delta': 0.002766546646878337, 'tau': 0.07391136216087091, 'leaf_prediction': 'nb', 'nb_threshold': 100, 'drift_window_threshold': 137, 'switch_significance': 0.16471007955550615, 'min_branch_fraction': 0.08068997461021792, 'max_share_to_split': 0.5022727638575484, 'adwin_delta': 0.046119624463481, 'adwin_clock': 26, 'adwin_max_buckets': 10, 'adwin_min_window_length': 9, 'adwin_grace_period': 21}. Best is trial 100 with value: 26.520000000000024.


Precision:Precision: 34.08%
Positive Predictions:214.0
Average Profit:0.3461214953271029//Average Loss:-0.22634615384615414


In [ ]:
from stockml.optimizations.model_tuning import show_optuna_results
#Now we display the top 25 trials in terms of rewards and return a dataframe that contains details about the best runs
results_df = show_optuna_results(study2, 0, 25)
results_df

Trial 100 // Reward: 26.520000000000024
Trial 108 // Reward: 24.08000000000007
Trial 20 // Reward: 23.589999999999975
Trial 331 // Reward: 23.149999999999878
Trial 354 // Reward: 22.519999999999953
Trial 163 // Reward: 22.480000000000032
Trial 211 // Reward: 22.2500000000001
Trial 360 // Reward: 21.040000000000134
Trial 160 // Reward: 20.770000000000067
Trial 190 // Reward: 20.710000000000107
Trial 350 // Reward: 20.3100000000001
Trial 436 // Reward: 20.049999999999955
Trial 98 // Reward: 19.830000000000084
Trial 237 // Reward: 19.490000000000023
Trial 234 // Reward: 19.320000000000064
Trial 479 // Reward: 19.32000000000005
Trial 265 // Reward: 19.22999999999996
Trial 78 // Reward: 19.199999999999918
Trial 200 // Reward: 19.150000000000077
Trial 67 // Reward: 18.950000000000102
Trial 482 // Reward: 18.869999999999962
Trial 476 // Reward: 18.849999999999966
Trial 488 // Reward: 18.830000000000084
Trial 261 // Reward: 18.570000000000064
Trial 84 // Reward: 18.379999999999953


,trial_num,trial_rewards,trial_metric,trial_num_preds,trial_ave_profit,trial_ave_loss,trial_params
0,100,26.52,Precision: 38.29%,268.0,0.3491,-0.2287,"{'grace_period': 117, 'split_criterion': 'gini..."
1,108,24.08,Precision: 35.73%,273.0,0.3581,-0.2273,"{'grace_period': 81, 'split_criterion': 'gini'..."
2,20,23.59,Precision: 35.43%,237.0,0.3615,-0.2150,"{'grace_period': 112, 'split_criterion': 'gini..."
3,331,23.15,Precision: 35.75%,242.0,0.3585,-0.2252,"{'grace_period': 58, 'split_criterion': 'gini'..."
4,354,22.52,Precision: 37.15%,237.0,0.3605,-0.2297,"{'grace_period': 50, 'split_criterion': 'info_..."
5,163,22.48,Precision: 34.67%,242.0,0.3666,-0.2210,"{'grace_period': 234, 'split_criterion': 'gini..."
6,211,22.25,Precision: 36.62%,234.0,0.3693,-0.2244,"{'grace_period': 104, 'split_criterion': 'info..."
7,360,21.04,Precision: 35.94%,202.0,0.3869,-0.2302,"{'grace_period': 67, 'split_criterion': 'gini'..."
8,160,20.77,Precision: 35.78%,263.0,0.3578,-0.2312,"{'grace_period': 107, 'split_criterion': 'info..."
9,190,20.71,Precision: 38.00%,255.0,0.3392,-0.2314,"{'grace_period': 231, 'split_criterion': 'hell..."


In [ ]:
#We can check the parameters of the best trial as well
study2.best_params

{'grace_period': 117,
 'split_criterion': 'gini',
 'delta': 0.02934919086783071,
 'tau': 0.03420910250735763,
 'leaf_prediction': 'nb',
 'nb_threshold': 89,
 'drift_window_threshold': 177,
 'switch_significance': 0.01642919578646868,
 'min_branch_fraction': 0.020437890996069415,
 'max_share_to_split': 0.5144247690012861,
 'adwin_delta': 0.06678408825164404,
 'adwin_clock': 42,
 'adwin_max_buckets': 7,
 'adwin_min_window_length': 16,
 'adwin_grace_period': 43}

In [ ]:
#Initiate somem variables that will be needed before we can store all the results in an SQL Table
model = 'BaggingClassifier(HoeffdingAdaptiveTreeClassifier)'
source = 'ibapi'
timeframe = '30m'
trial_start = X.index[0]
trial_end = X.index[-1]


results_df['ml_model'] = model
results_df['study_datetime'] =current_time
results_df['ticker'] = ticker
results_df['source'] = source
results_df['timeframe'] = timeframe
results_df['trial_start'] = trial_start
results_df['trial_end'] = trial_end
results_df['trial_metric'] = results_df['trial_metric'].astype(str)

results_df['trial_id'] = (
    results_df['study_datetime'].astype(str) + ' ' +
    results_df['ticker'].astype(str) + ' ' +
    results_df['trial_num'].astype(str) + ' ' +
    results_df['trial_rewards'].astype(str)
)

%store results_df
%store study2

results_df

,trial_num,trial_rewards,trial_metric,trial_num_preds,trial_ave_profit,trial_ave_loss,trial_params,ml_model,study_datetime,ticker,source,timeframe,trial_start,trial_end,trial_id
0,100,26.52,Precision: 38.29%,268.0,0.3491,-0.2287,"{'grace_period': 117, 'split_criterion': 'gini...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 100 26.52
1,108,24.08,Precision: 35.73%,273.0,0.3581,-0.2273,"{'grace_period': 81, 'split_criterion': 'gini'...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 108 24.08
2,20,23.59,Precision: 35.43%,237.0,0.3615,-0.2150,"{'grace_period': 112, 'split_criterion': 'gini...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 20 23.59
3,331,23.15,Precision: 35.75%,242.0,0.3585,-0.2252,"{'grace_period': 58, 'split_criterion': 'gini'...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 331 23.15
4,354,22.52,Precision: 37.15%,237.0,0.3605,-0.2297,"{'grace_period': 50, 'split_criterion': 'info_...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 354 22.52
5,163,22.48,Precision: 34.67%,242.0,0.3666,-0.2210,"{'grace_period': 234, 'split_criterion': 'gini...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 163 22.48
6,211,22.25,Precision: 36.62%,234.0,0.3693,-0.2244,"{'grace_period': 104, 'split_criterion': 'info...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 211 22.25
7,360,21.04,Precision: 35.94%,202.0,0.3869,-0.2302,"{'grace_period': 67, 'split_criterion': 'gini'...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 360 21.04
8,160,20.77,Precision: 35.78%,263.0,0.3578,-0.2312,"{'grace_period': 107, 'split_criterion': 'info...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 160 20.77
9,190,20.71,Precision: 38.00%,255.0,0.3392,-0.2314,"{'grace_period': 231, 'split_criterion': 'hell...",BaggingClassifier(HoeffdingAdaptiveTreeClassif...,2024-11-16 14:26:00,UPRO,ibapi,30m,2024-08-20 02:30:00,2024-11-15 09:30:00,2024-11-16 14:26:00 UPRO 190 20.71


In [ ]:
import sys
#Finally we store the results to an SQL Table
del sys.modules['stockml.sql']
import stockml.sql as ss

ss.write_optuna_results(results_df, ss.StockOptunaResults30m)

Optuna results written successfully.


In [121]:
param_importances2 = optuna.visualization.plot_param_importances(study2)
param_importances2.show()

In [ ]:
#We change the values of some parameters again based on our previous results and delete ones that had low importance 
new_params = {'leaf_prediction':('categorical', ['nb']),
              'grace_period': ('int', (10, 200)),
              'tau': ('uniform', (0.001, 0.075)),
              'max_share_to_split': ('uniform', (0.1, 0.7)),
              'drift_window_threshold': ('int', (10, 200)),
              'min_branch_fraction': ('uniform', (0.001, 0.05)),
              'nb_threshold': ('int', (50, 150))
}

exclude_params = ['bootstrap_sampling', 'remove_poor_attrs', 'merit_preprune', 'binary_split']


now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M")
current_time = datetime.strptime(current_time, "%Y-%m-%d %H:%M")

#Run optuna again with the new parameter search spaces
from stockml.optimizations.model_tuning import optuna_classification_reward
study3 = optuna_classification_reward(X_data=X, 
                                      y_data=y,
                                      model=model_name, 
                                      n_trials=500, 
                                      param_ranges=new_params, exclude_params=exclude_params)

[I 2024-11-16 17:30:22,231] A new study created in memory with name: no-name-7b46c319-c11a-4b45-8142-369f53df450b
[I 2024-11-16 17:30:25,934] Trial 0 finished with value: -0.06000000000010175 and parameters: {'grace_period': 140, 'split_criterion': 'hellinger', 'delta': 2.1517332599813074e-07, 'tau': 0.018209144698950522, 'leaf_prediction': 'nb', 'nb_threshold': 97, 'drift_window_threshold': 82, 'switch_significance': 0.034402337489151995, 'min_branch_fraction': 0.042718123883734437, 'max_share_to_split': 0.3556721034320689, 'adwin_delta': 0.0003011886107671482, 'adwin_clock': 50, 'adwin_max_buckets': 3, 'adwin_min_window_length': 19, 'adwin_grace_period': 44}. Best is trial 0 with value: -0.06000000000010175.


Precision:Precision: 33.52%
Positive Predictions:242.0
Average Profit:0.31508264462809893//Average Loss:-0.23801749271137038


[I 2024-11-16 17:30:29,294] Trial 1 finished with value: 5.199999999999989 and parameters: {'grace_period': 106, 'split_criterion': 'info_gain', 'delta': 0.0003901383081224973, 'tau': 0.05866565066365629, 'leaf_prediction': 'nb', 'nb_threshold': 135, 'drift_window_threshold': 26, 'switch_significance': 0.11289459609041833, 'min_branch_fraction': 0.008654491721116117, 'max_share_to_split': 0.5186844088648939, 'adwin_delta': 0.01109852056729204, 'adwin_clock': 17, 'adwin_max_buckets': 10, 'adwin_min_window_length': 19, 'adwin_grace_period': 19}. Best is trial 1 with value: 5.199999999999989.


Precision:Precision: 33.25%
Positive Predictions:251.0
Average Profit:0.3323107569721119//Average Loss:-0.23197222222222258


[I 2024-11-16 17:30:35,147] Trial 2 finished with value: 14.470000000000098 and parameters: {'grace_period': 92, 'split_criterion': 'hellinger', 'delta': 0.03208395207428351, 'tau': 0.028296042164932055, 'leaf_prediction': 'nb', 'nb_threshold': 148, 'drift_window_threshold': 149, 'switch_significance': 0.07379511778703272, 'min_branch_fraction': 0.013654440439078791, 'max_share_to_split': 0.6997743846359814, 'adwin_delta': 0.013941277218664694, 'adwin_clock': 1, 'adwin_max_buckets': 8, 'adwin_min_window_length': 20, 'adwin_grace_period': 14}. Best is trial 2 with value: 14.470000000000098.


...

Precision:Precision: 35.20%
Positive Predictions:220.0
Average Profit:0.3430454545454549//Average Loss:-0.22872852233677007


[I 2024-11-16 18:06:10,140] Trial 498 finished with value: 5.699999999999989 and parameters: {'grace_period': 27, 'split_criterion': 'hellinger', 'delta': 8.593178154393194e-06, 'tau': 0.05286919190258382, 'leaf_prediction': 'nb', 'nb_threshold': 133, 'drift_window_threshold': 22, 'switch_significance': 0.08500418515753286, 'min_branch_fraction': 0.006976385800051449, 'max_share_to_split': 0.6338623289966745, 'adwin_delta': 0.02256397707152999, 'adwin_clock': 22, 'adwin_max_buckets': 9, 'adwin_min_window_length': 18, 'adwin_grace_period': 43}. Best is trial 167 with value: 28.210000000000093.


Precision:Precision: 34.93%
Positive Predictions:255.0
Average Profit:0.31007843137254926//Average Loss:-0.23117647058823554


[I 2024-11-16 18:06:13,884] Trial 499 finished with value: 15.410000000000025 and parameters: {'grace_period': 103, 'split_criterion': 'gini', 'delta': 1.0315167840986964e-06, 'tau': 0.05841497002836517, 'leaf_prediction': 'nb', 'nb_threshold': 103, 'drift_window_threshold': 84, 'switch_significance': 0.08246589738082125, 'min_branch_fraction': 0.030681813559638674, 'max_share_to_split': 0.6650020741961873, 'adwin_delta': 0.0334706899898194, 'adwin_clock': 62, 'adwin_max_buckets': 5, 'adwin_min_window_length': 2, 'adwin_grace_period': 25}. Best is trial 167 with value: 28.210000000000093.


Precision:Precision: 35.41%
Positive Predictions:261.0
Average Profit:0.33375478927203106//Average Loss:-0.22695266272189393


In [ ]:

#And finally we store the results to an SQL Table
from stockml.optimizations.model_tuning import show_optuna_results

results_df2 = show_optuna_results(study3, 0, 25)
results_df2

,trial_num,trial_rewards,trial_metric,trial_num_preds,trial_ave_profit,trial_ave_loss,trial_params
0,167,28.21,Precision: 35.71%,255.0,0.3705,-0.2189,"{'grace_period': 104, 'split_criterion': 'gini..."
1,279,25.37,Precision: 37.01%,245.0,0.3506,-0.2208,"{'grace_period': 105, 'split_criterion': 'info..."
2,144,24.63,Precision: 37.02%,214.0,0.3840,-0.2354,"{'grace_period': 114, 'split_criterion': 'info..."
3,474,24.48,Precision: 36.67%,238.0,0.3455,-0.2153,"{'grace_period': 111, 'split_criterion': 'gini..."
4,447,24.42,Precision: 36.96%,272.0,0.3582,-0.2340,"{'grace_period': 78, 'split_criterion': 'gini'..."
5,189,23.46,Precision: 36.98%,260.0,0.3458,-0.2253,"{'grace_period': 115, 'split_criterion': 'gini..."
6,288,23.19,Precision: 36.26%,231.0,0.3610,-0.2187,"{'grace_period': 72, 'split_criterion': 'gini'..."
7,34,22.74,Precision: 36.26%,248.0,0.3554,-0.2291,"{'grace_period': 40, 'split_criterion': 'gini'..."
8,374,21.99,Precision: 34.67%,234.0,0.3849,-0.2374,"{'grace_period': 73, 'split_criterion': 'gini'..."
9,152,21.91,Precision: 35.18%,280.0,0.3678,-0.2384,"{'grace_period': 107, 'split_criterion': 'info..."
